In [1]:
import numpy as np
import scipy.sparse as sp

In [2]:
def readDataFromFile (fileName):
    "This functions reads data from a file and store it in two matrices"
    #Open the file
    file = open(fileName, 'r')
 
    #Now we have to read the first line and check if it's sparse or dense
    firstLine = file.readline()
    words = firstLine.split()
    word = words[1]
    if word[:-1] == 'SPARSE':
        sparse = True #The file is in sparse mode
    else:
        sparse = False #The file is in dense mode
 
 
    secondLine = file.readline()
    words = secondLine.split()
    instances = int(words[1])
    thirdLine = file.readline()
    words = thirdLine.split()
    attributes = int(words[1])
    fourthLine = file.readline()
    words = fourthLine.split()
    labels = int(words[1])
    #Now we do a loop reading all the other lines
    #Then we read the file, different way depending if sparse or dense
 
    #The loop starts in the first line of data
    #We have to store that data in two matrices
    X = np.zeros((instances, attributes), dtype=float)
    y = np.zeros((instances, labels), dtype=int)
    numberLine = 0
    for line in file.readlines():
        putToX = True
        firstIndex = 1
        numberData = 0
        numberY = 0
        for data in line.split():
            if sparse:#Sparse format, we have to split each data
                if data == '[':
                    putToX = False
 
                if putToX == True and (data != '[' and data != ']'):
                    sparseArray = data.split(':')
                    lastIndex = int(sparseArray[0])
                    for i in range(firstIndex, lastIndex - 1):
                        X[numberLine, i-1] = float(0)
                    X[numberLine, lastIndex-1] = float(sparseArray[1])
                    firstIndex = lastIndex-1
                else:
                    if (data != '[') and (data != ']'):
                        aux = float(data)
                        y[numberLine, numberY] = int(aux)
                        numberY += 1
               
            else:#Dense format
                if data == '[':
                    putToX = False
 
                if putToX == True and (data != '[' and data != ']'):
                    X[numberLine, numberData] = float(data)
                else:
                    if (data != '[') and (data != ']'):
                        #This is good for the dense format
                        aux = float(data)
                        y[numberLine, numberY] = int(aux)
                        numberY += 1
            numberData += 1
       
        numberLine += 1
    X = sp.csr_matrix(X)
    file.close()
    return X, y

In [3]:
#Sacamos los datos de train y test generados (.train y .gen)

In [4]:
#X_train.shape[0]
#y_train.shape[1]
#---------------
#X_test.shape[0]
#y_test.shape[1]

In [5]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics
#CHANGE DATASET NAME HERE:
dataset = 'yeast'

In [6]:
nfolds=10
for i in range(0, nfolds):
    X_train,y_train=readDataFromFile('./datasets/'+dataset+'/'+dataset+str(i)+'.rtrain')
    print('Reading: ./datasets/'+dataset+'/'+dataset+str(i)+'.rtrain')
    X_test,y_test=readDataFromFile('./datasets/'+dataset+'/'+dataset+str(i)+'.rtest')
    print('Reading: ./datasets/'+dataset+'/'+dataset+str(i)+'.rtest')
    classif = BinaryRelevance(classifier=RandomForestClassifier(n_estimators=10),require_dense=[False,True])
    classif.fit(X_train,y_train)
    y_score = classif.predict(X_test)
    y_prob = classif.predict_proba(X_test)
    fold_accuracy = []
    fold_hamming = []
    fold_cover = []
    fold_rank = []
    fold_prec = []
    fold_auc = []
    folds = []
    desired_number = []
    #-----------------------------------------#
    #Medidas: sklearn.metrics...(true,predict,..)
    acc= sklearn.metrics.accuracy_score(y_test, y_score)
    fold_accuracy.append(acc)
    print ("Accuracy: %0.5f"%acc)
    #-----------------------------------------#
    hl=sklearn.metrics.hamming_loss(y_test, y_score)
    fold_hamming.append(hl)
    print("Hamming loss: %0.5f"%hl)
    #-----------------------------------------#
    #Coverage
    c=sklearn.metrics.coverage_error(y_test, y_prob.toarray(), sample_weight=None)
    fold_cover.append(c)
    print ("Coverage: %0.5f - 1"%c)
    #-----------------------------------------#
    #Ranking loss
    rl=sklearn.metrics.label_ranking_loss(y_test, y_prob.toarray(), sample_weight=None)
    fold_rank.append(rl)
    print("Ranking loss: %0.5f"%rl)
    #-----------------------------------------#
    #Mean average precision
    m=sklearn.metrics.average_precision_score(y_test, y_score.toarray(), average='macro', pos_label=1, sample_weight=None)
    fold_prec.append(m)
    print("Mean average precision: %0.5f"%m)
    #-----------------------------------------#
    #Micro-average AUC
    rmi=sklearn.metrics.roc_auc_score(y_test, y_score.toarray(), average='micro', sample_weight=None, max_fpr=None)
    fold_auc.append(rmi)
    print("ROC AUC micro: %0.5f"%rmi)
    print("-------------------")
print("Problems while trying to calculate coverage & ranking loss due to probab measure")

Reading: ./datasets/yeast/yeast0.rtrain
Reading: ./datasets/yeast/yeast0.rtest
Accuracy: 0.11157
Hamming loss: 0.19421
Coverage: 8.16942 - 1
Ranking loss: 0.19723
Mean average precision: 0.35592
ROC AUC micro: 0.72852
-------------------
Reading: ./datasets/yeast/yeast1.rtrain
Reading: ./datasets/yeast/yeast1.rtest
Accuracy: 0.10331
Hamming loss: 0.21488
Coverage: 8.66116 - 1
Ranking loss: 0.22192
Mean average precision: 0.37404
ROC AUC micro: 0.70630
-------------------
Reading: ./datasets/yeast/yeast2.rtrain
Reading: ./datasets/yeast/yeast2.rtest
Accuracy: 0.11157
Hamming loss: 0.21488
Coverage: 8.57851 - 1
Ranking loss: 0.22515
Mean average precision: 0.37146
ROC AUC micro: 0.70327
-------------------
Reading: ./datasets/yeast/yeast3.rtrain
Reading: ./datasets/yeast/yeast3.rtest
Accuracy: 0.11157
Hamming loss: 0.21635
Coverage: 8.67355 - 1
Ranking loss: 0.22963
Mean average precision: 0.35963
ROC AUC micro: 0.70412
-------------------
Reading: ./datasets/yeast/yeast4.rtrain
Reading:

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/ranking.py:526: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


Reading: ./datasets/yeast/yeast6.rtrain
Reading: ./datasets/yeast/yeast6.rtest
Accuracy: 0.09504
Hamming loss: 0.21458
Coverage: 8.32645 - 1
Ranking loss: 0.23351
Mean average precision: 0.33892
ROC AUC micro: 0.69770
-------------------
Reading: ./datasets/yeast/yeast7.rtrain
Reading: ./datasets/yeast/yeast7.rtest
Accuracy: 0.10788
Hamming loss: 0.20717
Coverage: 8.46473 - 1
Ranking loss: 0.22530
Mean average precision: 0.36388
ROC AUC micro: 0.71364
-------------------
Reading: ./datasets/yeast/yeast8.rtrain
Reading: ./datasets/yeast/yeast8.rtest
Accuracy: 0.11618
Hamming loss: 0.20065
Coverage: 8.24481 - 1
Ranking loss: 0.21203
Mean average precision: 0.37336
ROC AUC micro: 0.72078
-------------------
Reading: ./datasets/yeast/yeast9.rtrain
Reading: ./datasets/yeast/yeast9.rtest
Accuracy: 0.12033
Hamming loss: 0.21162
Coverage: 8.30705 - 1
Ranking loss: 0.22802
Mean average precision: 0.37211
ROC AUC micro: 0.70574
-------------------
Problems while trying to calculate coverage & ra

In [7]:
#yscore = classif.score(X_test, y_test)

In [8]:
#y_score #prediccion de y en base al clasificador

In [9]:
print ("Avg Accuracy: ")
print sum(fold_accuracy)/len(fold_accuracy)

Avg Accuracy: 
0.12033195020746888


In [10]:
print("Hamming loss: ")
print sum(fold_hamming)/len(fold_hamming)

Hamming loss: 
0.211618257261


In [11]:
print ("Coverage: ")
print (sum(fold_cover)/len(fold_cover))-1

Coverage: 
7.307053941908714


In [12]:
print("Ranking loss: ")
print sum(fold_rank)/len(fold_rank)

Ranking loss: 
0.22801611547906747


In [13]:
print("Mean average precision: ")
print sum(fold_prec)/len(fold_prec)

Mean average precision: 
0.3721111735529662


In [14]:
print("Micro average AUC: ")
print sum(fold_auc)/len(fold_auc)

Micro average AUC: 
0.7057367411349712


In [15]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_score))

              precision    recall  f1-score   support

           0       0.80      0.36      0.50        88
           1       0.61      0.49      0.55       104
           2       0.80      0.56      0.66        98
           3       0.79      0.36      0.49        87
           4       0.96      0.30      0.46        82
           5       0.76      0.20      0.32        65
           6       0.75      0.14      0.24        42
           7       0.67      0.08      0.14        50
           8       0.00      0.00      0.00        16
           9       0.00      0.00      0.00        24
          10       0.00      0.00      0.00        25
          11       0.66      0.91      0.76       158
          12       0.68      0.90      0.78       157
          13       0.00      0.00      0.00         5

   micro avg       0.70      0.50      0.58      1001
   macro avg       0.53      0.31      0.35      1001
weighted avg       0.68      0.50      0.53      1001
 samples avg       0.67   

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
